# Keras

In [ ]:
# Run this in Jupyter's Terminal
# pip install tensorflow

In [ ]:
import pandas
import numpy as np
from sklearn.metrics import f1_score,confusion_matrix

import mlflow

import tensorflow
from tensorflow import keras
import mlflow.keras

from data_utils import get_train_test_split_for_stock
from config import *

# Retrieve data

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_split_for_stock(PATH_TO_DATA_FILE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Set experiment

In [ ]:
mlflow.set_experiment("SP_EXP_Modelling")
mlflow.tensorflow.autolog()

# Create model

In [ ]:
model = keras.Sequential([
  keras.layers.Dense(
    units=36,
    activation='relu',
    input_shape=(X_train.shape[-1],)
  ),
  keras.layers.BatchNormalization(),
  keras.layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.001),
  loss="binary_crossentropy",
  metrics="Accuracy"
)

# Run the model

In [ ]:
with mlflow.start_run(run_name='keras') as run:
    model.fit(
        X_train,
        y_train,
        epochs=20,
        validation_split=0.05,
        shuffle=True,
        verbose=0
    )
    preds = model.predict(X_test)
    y_pred = np.where(preds > 0.5, 1, 0)
    f1 = f1_score(y_test, y_pred)
    
    mlflow.set_tag("Model_name", "keras")
    mlflow.log_metric(key="testing_f1_score", value=f1)

In [ ]:
f1

# Show
- conda.yaml for the model
- compare models -> "testing_f1_score"